### 변수 명 앞에 _를 붙이는 경우 ?

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FD:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]


In [2]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

In [3]:
deck = FD()
len(deck)

52

In [4]:
deck[0], deck[-1]

(Card(rank='2', suit='spades'), Card(rank='A', suit='hearts'))

In [5]:
from random import choice

choice(deck), choice(deck), choice(deck)

(Card(rank='6', suit='hearts'),
 Card(rank='8', suit='hearts'),
 Card(rank='K', suit='clubs'))

In [6]:
print(deck[:3])
# 스페이드 A부터 시작해서 13개씩 건너뛰어 A만 출력
print(deck[12::13])

[Card(rank='2', suit='spades'), Card(rank='3', suit='spades'), Card(rank='4', suit='spades')]
[Card(rank='A', suit='spades'), Card(rank='A', suit='diamonds'), Card(rank='A', suit='clubs'), Card(rank='A', suit='hearts')]


In [7]:
for card in deck[:5]:
    print(card)

print('-'*50)
# reversed = 뒤집기
for card in reversed(deck[:5]):
    print(card)

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')
Card(rank='5', suit='spades')
Card(rank='6', suit='spades')
--------------------------------------------------
Card(rank='6', suit='spades')
Card(rank='5', suit='spades')
Card(rank='4', suit='spades')
Card(rank='3', suit='spades')
Card(rank='2', suit='spades')


In [8]:
# __contains__() 메서드가 있어 in 연산자 작동
Card('Q', 'hearts') in deck

True

A가 제일 높고 2가 제일 낮다.   
스페이드, 하트, 다이아몬드, 클로버 순으로 랭크가 높다.

In [9]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)
# key 함수
def spades_high(card):
    # 숫자 순서 (2->0, J->9, A->12)
    rank_v = FD.ranks.index(card.rank)
    # 같은 숫자끼리 비교 가능하게 padding
    return rank_v * len(suit_values) + suit_values[card.suit]

In [10]:
# key 함수의 return 값을 기준으로 오름차순 정렬한다.
for card in sorted(deck, key=spades_high)[:10]:
    print(card)

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='diamonds')


FD는 암묵적으로 object를 상속받지만, __len__() 과 __getitem__() 특별 메서드를 구현함으로써   
표준 파이썬 시퀀스처럼 작동하므로 예제의 라이브러리와 핵심 언어기능을 사용할 수 있다. ?
   
__len__(), __getitem__() 메서드는 모든 작업을 list 객체인 self._cards에 떠넘길 수 있다 ?

### 특별 메서드

In [11]:
from math import hypot

class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    def __repr__(self) -> str:
        return f'Vector({self.x}, {self.y})'
    def __abs__(self):
        return hypot(self.x, self.y)
    def __bool__(self):
        return bool(abs(self))
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

In [12]:
v1 = Vector(2, 4)
v2 = Vector(2, 1)
f'v1 = {v1}, v1 + v2 = {v1 + v2}, abs(v1) = {abs(v1)}, v1 * 3 = {v1 * 3}, bool(v1) = {bool(v1)}'

'v1 = Vector(2, 4), v1 + v2 = Vector(4, 5), abs(v1) = 4.47213595499958, v1 * 3 = Vector(6, 12), bool(v1) = True'

__repr__메서드를 구현하지 않으면 Vector 객체는 콘솔에 <Vector 어쩌구 >와 같은 형태로 출력됩니다.   
__bool__이나 __len__을 구현하지 않은 경우, 기본적으로 참이라고 간주합니다.   
__bool__이 구현되어 있지 않으면 파이썬은 x.\_\_len\_\_()을 호출합니다.

### len()이 메서드가 아닌 이유   
__실용성이 순수성에 우선한다.__ 에 따라 len은 아주 많이 사용되므로 내장 객체에 대해서는 메서드를 호출하지 않고   
단지 C언어 구조체의 필드를 읽어옴으로써 효율적이게 작동합니다.